In [1]:
# Nuestro path de inicio siempre debe ser el home dir, e
import os
os.chdir(os.path.abspath(".."))

# Imports
import json
import pandas as pd
import stable_whisper
import warnings
from src.votaciones import votacion_promedio_ponderada

# ignoramos warnings
warnings.filterwarnings('ignore')

In [2]:
# Archivos importantes y paths
df_annotations = pd.read_excel('data/annotations.xlsx')

# Omitimos todos los audios en development
df_annotations = df_annotations[df_annotations['Type'] != 'Development'].reset_index(drop = True)

In [3]:
# Obtenemos el json con los intervalos
with open('data/intervalos_transcripciones_whisper.json', 'r') as f:
    intervalos_transcripciones = json.load(f)
    
# Comprabamos seguimos teniendo los 244 audios
len(intervalos_transcripciones.keys())

# Obtenemos el json con los pesos
with open('data/pesos_anotadores_dif.json', 'r') as f:
    pesos_anotadores = json.load(f)

In [4]:
# Audios que no estan en development, deberían ser 205
lista_audios = df_annotations['Audio_Name'].unique()
len(lista_audios)

205

In [5]:
# Creo mi diccionario que voy a guardare como json
targets_mean_vote = {}

# Loop por cada audio en la lista de audios que no estan en development
for audio_name in lista_audios:
    
    print('Procesando el audio:', audio_name)
    
    df_audio = df_annotations[df_annotations['Audio_Name'] == audio_name]
    partes = df_audio['Part_Num'].max()
    
    # Inicializo valores para el audio en el diccionario
    targets_mean_vote[audio_name] = {}
    targets_mean_vote[audio_name]['rangos'] = []
    targets_mean_vote[audio_name]['targets'] = []
    targets_mean_vote[audio_name]['indice'] = []
    
    # Loop por cada parte del audio
    for part_num in range(partes):
        part_num = part_num + 1
        
        print(f'Procesando parte {part_num} de {audio_name}')
        
        # Obtengo votación para la parte del audio
        df_votacion = votacion_promedio_ponderada(df_annotations, 
                                                  pesos_anotadores, 
                                                  audio_name = audio_name,
                                                  part_num = part_num, 
                                                  multiplicador = 3)
        
        # Verificamos que solo halla un start time y nos lo quedamos
        assert len(df_audio[df_audio['Part_Num'] == part_num]['start_time'].unique()) == 1, 'Más de un start time'
        start_time = df_audio[df_audio['Part_Num'] == part_num]['start_time'].unique()[0]
        
        # Agregamos al audio el start time necesario
        df_votacion['Time'] = df_votacion['Time'] + start_time
        
        # Loop principal para cargar las votaciones promedio
        i = 0
        
        int_act = intervalos_transcripciones[audio_name]['rangos']
        ts_primera_anotacion = df_votacion['Time'][0]
        ts_ultima_anotacion = df_votacion['Time'][len(df_votacion) - 1]
        
        # Agrego ventana de 0.1 segundos por lo visto en el análisis en la notebook 03
        while i + 1 <= len(int_act) and int_act[i][1] + 0.1 < ts_ultima_anotacion:
            
            if int_act[i][0] > ts_primera_anotacion:
                inicio = int_act[i][0]
                fin = int_act[i][1] + 0.1
                
                votacion_promedio = df_votacion[(df_votacion['Time'] >= inicio) & (df_votacion['Time'] <= fin)][['Valence','Arousal','Dominance']].mean().values.tolist()
                
                targets_mean_vote[audio_name]['targets'].append(votacion_promedio)
                targets_mean_vote[audio_name]['rangos'].append((inicio, fin))
                targets_mean_vote[audio_name]['indice'].append(i)
            
            i += 1
    
    assert len(targets_mean_vote[audio_name]['targets']) == len(targets_mean_vote[audio_name]['rangos']), 'Rangos no coinciden'
        
    print(f'{audio_name} procesado con éxito')
    print('')
    print('---------------------------------------------------------')

Procesando el audio: MSP-Conversation_0021.wav
Procesando parte 1 de MSP-Conversation_0021.wav
Procesando parte 2 de MSP-Conversation_0021.wav
Procesando parte 3 de MSP-Conversation_0021.wav
Procesando parte 4 de MSP-Conversation_0021.wav
MSP-Conversation_0021.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0023.wav
Procesando parte 1 de MSP-Conversation_0023.wav
Procesando parte 2 de MSP-Conversation_0023.wav
MSP-Conversation_0023.wav procesado con éxito

---------------------------------------------------------
Procesando el audio: MSP-Conversation_0035.wav
Procesando parte 1 de MSP-Conversation_0035.wav


KeyboardInterrupt: 

Validaciones

In [6]:
print(len(targets_mean_vote.keys()), '/ 205') # cantidad de audios en development
print(targets_mean_vote.keys())

205 / 205
dict_keys(['MSP-Conversation_0021.wav', 'MSP-Conversation_0023.wav', 'MSP-Conversation_0035.wav', 'MSP-Conversation_0047.wav', 'MSP-Conversation_0061.wav', 'MSP-Conversation_0079.wav', 'MSP-Conversation_0083.wav', 'MSP-Conversation_0087.wav', 'MSP-Conversation_0088.wav', 'MSP-Conversation_0103.wav', 'MSP-Conversation_0114.wav', 'MSP-Conversation_0125.wav', 'MSP-Conversation_0135.wav', 'MSP-Conversation_0147.wav', 'MSP-Conversation_0153.wav', 'MSP-Conversation_0160.wav', 'MSP-Conversation_0166.wav', 'MSP-Conversation_0167.wav', 'MSP-Conversation_0172.wav', 'MSP-Conversation_0180.wav', 'MSP-Conversation_0190.wav', 'MSP-Conversation_0201.wav', 'MSP-Conversation_0202.wav', 'MSP-Conversation_0218.wav', 'MSP-Conversation_0227.wav', 'MSP-Conversation_0231.wav', 'MSP-Conversation_0257.wav', 'MSP-Conversation_0269.wav', 'MSP-Conversation_0281.wav', 'MSP-Conversation_0295.wav', 'MSP-Conversation_0300.wav', 'MSP-Conversation_0361.wav', 'MSP-Conversation_0380.wav', 'MSP-Conversation_0381

In [9]:
for i in targets_mean_vote.keys():
    print(i, len(targets_mean_vote[i]['rangos']))

MSP-Conversation_0021.wav 1359
MSP-Conversation_0023.wav 146
MSP-Conversation_0035.wav 199
MSP-Conversation_0047.wav 205
MSP-Conversation_0061.wav 150
MSP-Conversation_0079.wav 276
MSP-Conversation_0083.wav 169
MSP-Conversation_0087.wav 355
MSP-Conversation_0088.wav 247
MSP-Conversation_0103.wav 911
MSP-Conversation_0114.wav 337
MSP-Conversation_0125.wav 389
MSP-Conversation_0135.wav 293
MSP-Conversation_0147.wav 140
MSP-Conversation_0153.wav 542
MSP-Conversation_0160.wav 383
MSP-Conversation_0166.wav 261
MSP-Conversation_0167.wav 316
MSP-Conversation_0172.wav 400
MSP-Conversation_0180.wav 224
MSP-Conversation_0190.wav 281
MSP-Conversation_0201.wav 257
MSP-Conversation_0202.wav 119
MSP-Conversation_0218.wav 250
MSP-Conversation_0227.wav 268
MSP-Conversation_0231.wav 313
MSP-Conversation_0257.wav 220
MSP-Conversation_0269.wav 491
MSP-Conversation_0281.wav 383
MSP-Conversation_0295.wav 210
MSP-Conversation_0300.wav 320
MSP-Conversation_0361.wav 450
MSP-Conversation_0380.wav 181
MSP-Conve

In [8]:
# Guardo el diccionario en archivo json
with open('data/objetivos_por_voto_promedio_ponderado_whisper.json', 'w') as f:
    json.dump(targets_mean_vote, f)